In [1]:
import numpy as np
import pandas as pd
import time

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [5]:
# Seleniumをあらゆる環境で起動させるChromeオプション
options = Options()
options.add_argument('--disable-gpu');
options.add_argument('--disable-extensions');
options.add_argument('--proxy-server="direct://"');
options.add_argument('--proxy-bypass-list=*');
options.add_argument('--start-maximized');
# options.add_argument('--headless'); # ※ヘッドレスモードを使用する場合、コメントアウトを外す

In [6]:
DRIVER_PATH = '../chromedriver'
# DRIVER_PATH = '/Users/Kenta/Desktop/Selenium/chromedriver' # ローカル
# DRIVER_PATH = '/app/.chromedriver/bin/chromedriver'        # heroku

# ブラウザの起動
driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=options)

C:\Users\Takumi Inui\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [7]:
# Selenium 経由でブラウザを操作する
url = 'https://widget.oanda.jp/order-book'
driver.get(url)

In [8]:
import datetime 
tm = datetime.datetime.now()
tm = tm - datetime.timedelta(minutes=tm.minute % 10,seconds=tm.second,microseconds=tm.microsecond)
print(tm)

2021-04-16 22:20:00


In [9]:
li_currency = ["USD / JPY","USD / JPY","XAG / USD","XAU / USD","EUR / USD","EUR / JPY","GBP / JPY","AUD / JPY",
              "GBP / USD","AUD / USD","EUR / AUD","EUR / CHF","EUR / GBP","GBP / CHF","NZD / USD","USD / CAD","USD / CHF"]

In [10]:
def getSource(c1, c2):
    element = driver.find_element_by_link_text(c1)
    #画像のリンクをクリック
    element.click()
    time.sleep(1)
    element = driver.find_element_by_link_text(c2)
    #画像のリンクをクリック
    element.click()
    source = driver.page_source
    soup = BeautifulSoup(source, "html.parser") 
    x = soup.find_all("rect", class_="highcharts-point")
    li = []
    for i in x:
        li.append(int(i.get("height")))
    liFill = []
    for i in x:
        liFill.append(i.get("fill"))
    return li, liFill

In [11]:
li = []
liFill = []
for i in range(len(li_currency)-1):
    a, b = getSource(li_currency[i],li_currency[i+1])
    li.append(a)
    liFill.append(b)
    time.sleep(5)

In [24]:
def makeNewlist(x):
    newList = [0,0,0,0,0,0,0,0]
    newList[0] = x[0]
    newList[7] = x[4]
    newList[2] = x[0]
    newList[5] = x[4]
    newList[1] = x[1] - x[0]
    newList[6] = x[3] - x[4]
    newList[3] = newList[1]
    newList[4] = newList[6]
    return newList

In [26]:
spaceLi = []
for i in range(len(liFill)):
    numli = [1,0,0,0,0]
    num = 0
    for j in range(len(liFill[i])-1):
        if liFill[i][j] == liFill[i][j+1]:
            numli[num] += 1
        else:
            num += 1
            numli[num] += 1
    spaceLi.append(numli)
list(map(makeNewlist,spaceLi))

[[35, 35, 35, 35, 35, 35, 35, 35],
 [35, 35, 35, 35, 35, 35, 35, 35],
 [201, 181, 201, 181, 319, 162, 319, 162],
 [59, 57, 59, 57, 63, 43, 63, 43],
 [46, 47, 46, 47, 46, 47, 46, 47],
 [47, 46, 47, 46, 47, 25, 47, 25],
 [79, 80, 79, 80, 77, 70, 77, 70],
 [37, 37, 37, 37, 37, 36, 37, 36],
 [60, 60, 60, 60, 60, 59, 60, 59],
 [38, 38, 38, 38, 38, 38, 38, 38],
 [51, 52, 51, 52, 43, 51, 43, 51],
 [42, 42, 42, 42, 39, 36, 39, 36],
 [23, 23, 23, 23, 23, 23, 23, 23],
 [85, 85, 85, 85, 67, 78, 67, 78],
 [20, 20, 20, 20, 20, 20, 20, 20],
 [43, 42, 43, 42, 36, 42, 36, 42]]

[[35, 70, 70, 70, 35],
 [35, 70, 70, 70, 35],
 [201, 382, 500, 481, 162],
 [59, 116, 120, 106, 43],
 [46, 93, 93, 93, 47],
 [47, 93, 93, 72, 25],
 [79, 159, 157, 147, 70],
 [37, 74, 74, 73, 36],
 [60, 120, 120, 119, 59],
 [38, 76, 76, 76, 38],
 [51, 103, 95, 94, 51],
 [42, 84, 81, 75, 36],
 [23, 46, 46, 46, 23],
 [85, 170, 152, 145, 78],
 [20, 40, 40, 40, 20],
 [43, 85, 78, 78, 42]]

In [14]:
# ブラウザを終了する
driver.quit()